# テキスト生成アプリを作ろう

これまでのカリキュラムで、プロンプトなどの基本的な概念や「プロンプトエンジニアリング」と呼ばれる分野があることを学んできました。ChatGPT、Office 365、Microsoft Power Platform など、さまざまなツールでプロンプトを使って何かを達成することができます。

自分のアプリにこうした体験を加えるには、プロンプトやコンプリーションといった概念を理解し、使うライブラリを選ぶ必要があります。この章では、まさにその方法を学びます。

## はじめに

この章では、次のことを学びます。

- openaiライブラリとその基本的な概念について
- openaiを使ったテキスト生成アプリの作成
- プロンプト、temperature、トークンなどの概念を使ってテキスト生成アプリを作る方法

## 学習目標

このレッスンの終わりには、次のことができるようになります。

- テキスト生成アプリが何か説明できる
- openaiを使ってテキスト生成アプリを作れる
- トークン数やtemperatureを調整して、出力のバリエーションを変えられる

## テキスト生成アプリとは？

通常、アプリを作るときは次のようなインターフェースがあります。

- コマンドベース：コンソールアプリは、コマンドを入力してタスクを実行する典型的なアプリです。例えば、`git`はコマンドベースのアプリです。
- ユーザーインターフェース（UI）：ボタンをクリックしたり、テキストを入力したり、オプションを選択したりするグラフィカルなUI（GUI）を持つアプリもあります。

### コンソールやUIアプリの限界

コマンドベースのアプリと比べてみましょう。

- **制限がある**：好きなコマンドを入力できるわけではなく、アプリが対応しているコマンドしか使えません。
- **言語が限定される**：多言語対応のアプリもありますが、基本的には特定の言語向けに作られています。追加で他の言語をサポートすることもできますが、最初は限定的です。

### テキスト生成アプリのメリット

では、テキスト生成アプリは何が違うのでしょうか？

テキスト生成アプリでは、より柔軟に操作でき、コマンドや入力言語に縛られません。自然言語でアプリとやり取りできるのが特徴です。さらに、膨大な情報を学習したデータソースとやり取りするため、従来のアプリのようにデータベースの中身だけに制限されません。

### テキスト生成アプリで何が作れる？

作れるものはたくさんあります。例えば：

- **チャットボット**：会社や製品について質問に答えるチャットボットなどが考えられます。
- **ヘルパー**：LLMはテキストの要約や、テキストから洞察を得たり、履歴書などの文章を作成したりするのが得意です。
- **コードアシスタント**：使う言語モデルによっては、コードを書くのを手助けするアシスタントも作れます。GitHub CopilotやChatGPTなどを使ってコード作成をサポートすることもできます。

## どうやって始める？

LLMと連携する方法として、主に次の2つがあります。

- APIを使う：プロンプトを含めたWebリクエストを作成し、生成されたテキストを受け取ります。
- ライブラリを使う：API呼び出しをラップして、より簡単に使えるようにしてくれるライブラリを利用します。

## ライブラリ／SDK

LLMと連携するための有名なライブラリはいくつかあります。

- **openai**：モデルへの接続やプロンプト送信が簡単にできるライブラリです。

さらに、より高機能なライブラリもあります。

- **Langchain**：Python対応で有名なライブラリです。
- **Semantic Kernel**：Microsoft製で、C#、Python、Javaに対応しています。

## openaiで最初のアプリを作る

最初のアプリをどう作るか、必要なライブラリや手順を見ていきましょう。

### openaiのインストール

  > [!NOTE] CodespacesやDevcontainerでこのノートブックを実行している場合は、この手順は不要です

OpenAIやAzure OpenAIとやり取りするためのライブラリはたくさんあります。C#、Python、JavaScript、Javaなど、さまざまな言語で使うことも可能です。  
ここではPythonの`openai`ライブラリを使うので、`pip`でインストールします。

```bash
pip install openai
```

CodespacesやDev Containerでノートブックを実行していない場合は、[Python](https://www.python.org/)も自分のPCにインストールしてください。

### リソース作成とAPIキーの取得

まだの場合は、次の手順を行ってください。

- OpenAIでアカウントを作成する <https://platform.openai.com/signup>
- APIキーを取得する <https://platform.openai.com/api-keys>

>[!NOTE]
> APIキーはコードと分けて管理するのがおすすめです。環境変数を使うと便利です。
> - `.env`ファイルで環境変数`OPENAI_KEY`にAPIキーを設定しましょう。このコースの前の演習を終えていれば、すでに設定済みです。
> - APIキーは一度しか表示されません。必ず正しくコピーできているか確認してください。もしうまく動作しない場合は、キーを削除して新しく作り直しましょう。

### OpenAIの設定

OpenAIを使う場合、設定方法は次の通りです。

```python
client = OpenAI(
  api_key = os.environ['OPENAI_API_KEY']
  )

deployment = "gpt-3.5-turbo"
```

上記では次の設定をしています。

- `api_key`：OpenAIのダッシュボードで取得したAPIキー
- `deployment`：使いたいGPTのバージョン

> [!NOTE]
> `os.environ`は環境変数を読み取る関数です。`OPENAI_API_KEY`などの環境変数を読み込むのに使えます。

## テキスト生成

テキストを生成するには、`chat.completion`クラスを使います。例を見てみましょう。

```python
prompt = "Complete the following: Once upon a time there was a"

completion = client.chat.completions.create(model=deployment, messages=[{"role": "user", "content": prompt}])
print(completion.choices[0].message.content)
```

上記のコードでは、コンプリーションオブジェクトを作成し、使いたいモデルとプロンプトを渡しています。そして、生成されたテキストを表示しています。

### チャットコンプリーション

これまで`Completion`を使ってテキスト生成してきましたが、チャットボット向けには`ChatCompletion`というクラスもあります。使い方の例です。

```python
client = OpenAI(
  api_key = os.environ['OPENAI_API_KEY']
  )

deployment = "gpt-3.5-turbo"

completion = client.chat.completions.create(model=deployment, messages=[{"role": "user", "content": "Hello world"}])
print(completion.choices[0].message.content)
```

この機能については、次の章でさらに詳しく説明します。

## 演習 - はじめてのテキスト生成アプリ

OpenAIサービスのセットアップと設定方法を学んだので、いよいよ最初のテキスト生成アプリを作ってみましょう。アプリを作るには、次の手順に従ってください。


1. 仮想環境を作成し、openaiをインストールします：

  > [!NOTE] この手順は、CodespacesやDevcontainer内でこのノートブックを実行する場合は不要です


In [ ]:
# Create virtual environment
! python -m venv venv
# Activate virtual environment
! source venv/bin/activate
# Install openai package
! pip install openai

> [!NOTE]
> Windowsを使用している場合は、`source venv/bin/activate` の代わりに `venv\Scripts\activate` と入力してください。

> [!NOTE]
> OpenAIキーは https://platform.openai.com/settings/organization/api-keys にアクセスし、`API keys` を探すことで見つけられます。そこで新しいキーを作成し、すぐに値をコピーできます。


1. *app.py* ファイルを作成し、次のコードを入力してください:


In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv()

client = OpenAI(
  api_key = os.environ['OPENAI_API_KEY']
)

deployment = "gpt-3.5-turbo"

# add your completion code
prompt = "Complete the following: Once upon a time there was a"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages)

# print response
print(completion.choices[0].message.content)

```output
     very unhappy _____.

    Once upon a time there was a very unhappy mermaid.
    ```


## さまざまな用途に合わせたプロンプトの種類

これまでに、プロンプトを使ってテキストを生成する方法を見てきました。実際にプログラムを動かして、いろいろな種類のテキストを作れるようにもなりましたね。

プロンプトは、いろいろなタスクに使えます。例えば：

- **特定の種類のテキストを作る**。詩を作ったり、クイズの問題を作ったりできます。
- **情報を調べる**。例えば「Web開発でCORSとは何ですか？」のように、知りたいことをプロンプトで聞くことができます。
- **コードを生成する**。メールアドレスのバリデーションに使う正規表現を作ったり、Webアプリのようなプログラム全体を作ることもできます。

## より実用的な使い方：レシピジェネレーター

家にある材料で何か料理を作りたいとき、レシピが必要ですよね。レシピを探す方法としては、検索エンジンを使うこともできますが、LLMを使うこともできます。

例えば、こんなプロンプトを書けます：

> 「鶏肉、じゃがいも、にんじんを使った料理のレシピを5つ教えてください。各レシピごとに使う材料をすべて挙げてください。」

このプロンプトに対して、次のような返答が得られるかもしれません：

```output
1. Roasted Chicken and Vegetables: 
Ingredients: 
- 4 chicken thighs
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 2 tablespoons olive oil
- 2 cloves garlic, minced
- 1 teaspoon dried thyme
- 1 teaspoon dried oregano
- Salt and pepper, to taste

2. Chicken and Potato Stew: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 cloves garlic, minced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

3. Chicken and Potato Bake: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 2 cloves garlic, minced
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

4. Chicken and Potato Soup: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 cloves garlic, minced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 4 cups chicken broth
- Salt and pepper, to taste

5. Chicken and Potato Hash: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 2 cloves garlic, minced
- 1 teaspoon dried oregano
```

この結果はとても便利ですね。何を作ればいいか分かります。ここで、さらに便利にするための工夫としては：

- 苦手な食材やアレルギーのある食材を除外する
- 家にない材料があれば、買い物リストを作る

これらのケースに対応するため、追加のプロンプトを考えてみましょう：

> 「私はにんにくアレルギーなので、にんにくを使ったレシピは除外し、代わりになるものを使ってください。また、鶏肉、じゃがいも、にんじんはすでに家にあるので、それ以外の材料で買い物リストも作ってください。」

すると、新しい結果が得られます：

```output
1. Roasted Chicken and Vegetables: 
Ingredients: 
- 4 chicken thighs
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 2 tablespoons olive oil
- 1 teaspoon dried thyme
- 1 teaspoon dried oregano
- Salt and pepper, to taste

2. Chicken and Potato Stew: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

3. Chicken and Potato Bake: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

4. Chicken and Potato Soup: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 4 cups chicken broth
- Salt and pepper, to taste

5. Chicken and Potato Hash: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 1 teaspoon dried oregano

Shopping List: 
- Olive oil
- Onion
- Thyme
- Oregano
- Salt
- Pepper
```

これで、にんにくを使わない5つのレシピと、家にある材料を考慮した買い物リストが手に入りました。


## 演習 - レシピジェネレーターを作成しよう

シナリオを体験したので、今度はそのシナリオに合ったコードを書いてみましょう。以下の手順に従ってください。

1. 既存の *app.py* ファイルを使って始めましょう
1. `prompt` 変数を見つけて、次のコードに変更してください:


In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv()

client = OpenAI(
  api_key = os.environ['OPENAI_API_KEY']
)

deployment = "gpt-3.5-turbo"

prompt = "Show me 5 recipes for a dish with the following ingredients: chicken, potatoes, and carrots. Per recipe, list all the ingredients used"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages, max_tokens=600)

# print response
print(completion.choices[0].message.content)

もし今このコードを実行すると、次のような出力が表示されるはずです。

```output
-Chicken Stew with Potatoes and Carrots: 3 tablespoons oil, 1 onion, chopped, 2 cloves garlic, minced, 1 carrot, peeled and chopped, 1 potato, peeled and chopped, 1 bay leaf, 1 thyme sprig, 1/2 teaspoon salt, 1/4 teaspoon black pepper, 1 1/2 cups chicken broth, 1/2 cup dry white wine, 2 tablespoons chopped fresh parsley, 2 tablespoons unsalted butter, 1 1/2 pounds boneless, skinless chicken thighs, cut into 1-inch pieces
-Oven-Roasted Chicken with Potatoes and Carrots: 3 tablespoons extra-virgin olive oil, 1 tablespoon Dijon mustard, 1 tablespoon chopped fresh rosemary, 1 tablespoon chopped fresh thyme, 4 cloves garlic, minced, 1 1/2 pounds small red potatoes, quartered, 1 1/2 pounds carrots, quartered lengthwise, 1/2 teaspoon salt, 1/4 teaspoon black pepper, 1 (4-pound) whole chicken
-Chicken, Potato, and Carrot Casserole: cooking spray, 1 large onion, chopped, 2 cloves garlic, minced, 1 carrot, peeled and shredded, 1 potato, peeled and shredded, 1/2 teaspoon dried thyme leaves, 1/4 teaspoon salt, 1/4 teaspoon black pepper, 2 cups fat-free, low-sodium chicken broth, 1 cup frozen peas, 1/4 cup all-purpose flour, 1 cup 2% reduced-fat milk, 1/4 cup grated Parmesan cheese

-One Pot Chicken and Potato Dinner: 2 tablespoons olive oil, 1 pound boneless, skinless chicken thighs, cut into 1-inch pieces, 1 large onion, chopped, 3 cloves garlic, minced, 1 carrot, peeled and chopped, 1 potato, peeled and chopped, 1 bay leaf, 1 thyme sprig, 1/2 teaspoon salt, 1/4 teaspoon black pepper, 2 cups chicken broth, 1/2 cup dry white wine

-Chicken, Potato, and Carrot Curry: 1 tablespoon vegetable oil, 1 large onion, chopped, 2 cloves garlic, minced, 1 carrot, peeled and chopped, 1 potato, peeled and chopped, 1 teaspoon ground coriander, 1 teaspoon ground cumin, 1/2 teaspoon ground turmeric, 1/2 teaspoon ground ginger, 1/4 teaspoon cayenne pepper, 2 cups chicken broth, 1/2 cup dry white wine, 1 (15-ounce) can chickpeas, drained and rinsed, 1/2 cup raisins, 1/2 cup chopped fresh cilantro
```

> NOTE, あなたのLLMは非決定的なので、プログラムを実行するたびに異なる結果が得られる場合があります。

では、どのように改善できるか見ていきましょう。改善するためには、コードが柔軟であること、つまり材料やレシピの数を簡単に変更・拡張できるようにすることが大切です。


In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv()

client = OpenAI(
  api_key = os.environ['OPENAI_API_KEY']
)

deployment = "gpt-3.5-turbo"

no_recipes = input("No of recipes (for example, 5: ")

ingredients = input("List of ingredients (for example, chicken, potatoes, and carrots: ")

# interpolate the number of recipes into the prompt an ingredients
prompt = f"Show me {no_recipes} recipes for a dish with the following ingredients: {ingredients}. Per recipe, list all the ingredients used"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages, max_tokens=600)

# print response
print(completion.choices[0].message.content)

コードを試しに実行すると、次のようになります：

```output
No of recipes (for example, 5: 3
List of ingredients (for example, chicken, potatoes, and carrots: milk,strawberries

-Strawberry milk shake: milk, strawberries, sugar, vanilla extract, ice cubes
-Strawberry shortcake: milk, flour, baking powder, sugar, salt, unsalted butter, strawberries, whipped cream        
-Strawberry milk: milk, strawberries, sugar, vanilla extract
```

### フィルターと買い物リストの追加で改善する

これで、レシピを作成できる動作するアプリができました。ユーザーからの入力（レシピ数や使う材料）に柔軟に対応できます。

さらに改善するために、次の機能を追加したいと思います：

- **材料のフィルタリング**。苦手な材料やアレルギーのある材料を除外できるようにしたいです。この変更を実現するには、既存のプロンプトを編集し、最後にフィルター条件を追加します。例えば：

    ```python
    filter = input("Filter (for example, vegetarian, vegan, or gluten-free: ")

    prompt = f"Show me {no_recipes} recipes for a dish with the following ingredients: {ingredients}. Per recipe, list all the ingredients used, no {filter}"
    ```

    上記のように、プロンプトの最後に`{filter}`を追加し、ユーザーからフィルター値も受け取るようにします。

    実際にプログラムを実行したときの入力例は次のようになります：
    
    ```output    
    No of recipes (for example, 5: 3
    List of ingredients (for example, chicken, potatoes, and carrots: onion,milk
    Filter (for example, vegetarian, vegan, or gluten-free: no milk

    1. French Onion Soup

    Ingredients:
    
    -1 large onion, sliced
    -3 cups beef broth
    -1 cup milk
    -6 slices french bread
    -1/4 cup shredded Parmesan cheese
    -1 tablespoon butter
    -1 teaspoon dried thyme
    -1/4 teaspoon salt
    -1/4 teaspoon black pepper
    
    Instructions:
    
    1. In a large pot, sauté onions in butter until golden brown.
    2. Add beef broth, milk, thyme, salt, and pepper. Bring to a boil.
    3. Reduce heat and simmer for 10 minutes.
    4. Place french bread slices on soup bowls.
    5. Ladle soup over bread.
    6. Sprinkle with Parmesan cheese.
    
    2. Onion and Potato Soup
    
    Ingredients:
    
    -1 large onion, chopped
    -2 cups potatoes, diced
    -3 cups vegetable broth
    -1 cup milk
    -1/4 teaspoon black pepper
    
    Instructions:
    
    1. In a large pot, sauté onions in butter until golden brown.
    2. Add potatoes, vegetable broth, milk, and pepper. Bring to a boil.
    3. Reduce heat and simmer for 10 minutes.
    4. Serve hot.
    
    3. Creamy Onion Soup
    
    Ingredients:
    
    -1 large onion, chopped
    -3 cups vegetable broth
    -1 cup milk
    -1/4 teaspoon black pepper
    -1/4 cup all-purpose flour
    -1/2 cup shredded Parmesan cheese
    
    Instructions:
    
    1. In a large pot, sauté onions in butter until golden brown.
    2. Add vegetable broth, milk, and pepper. Bring to a boil.
    3. Reduce heat and simmer for 10 minutes.
    4. In a small bowl, whisk together flour and Parmesan cheese until smooth.
    5. Add to soup and simmer for an additional 5 minutes, or until soup has thickened.
    ```

    ご覧の通り、ミルクが含まれるレシピは除外されています。ただし、乳糖不耐症の場合はチーズも除外したいかもしれませんので、フィルター内容は明確に指定する必要があります。

    ```python
    
- **Produce a shopping list**. We want to produce a shopping list, considering what we already have at home.

    For this functionality, we could either try to solve everything in one prompt or we could split it up into two prompts. Let's try the latter approach. Here we're suggesting adding an additional prompt, but for that to work, we need to add the result of the former prompt as context to the latter prompt. 

    Locate the part in the code that prints out the result from the first prompt and add the following code below:
    
    ```python
    old_prompt_result = completion.choices[0].text
    prompt = "生成されたレシピの買い物リストを作成してください。すでに持っている材料は含めないでください。"
    
    new_prompt = f"{old_prompt_result} {prompt}"
    messages = [{"role": "user", "content": new_prompt}]
    completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=1200)
    
    # レスポンスを表示
    print("買い物リスト:")
    print(completion.choices[0].message.content)
    ```

    Note the following:

    - We're constructing a new prompt by adding the result from the first prompt to the new prompt: 
    
        ```python
        new_prompt = f"{old_prompt_result} {prompt}"
        messages = [{"role": "user", "content": new_prompt}]
        ```

    - We make a new request, but also considering the number of tokens we asked for in the first prompt, so this time we say `max_tokens` is 1200. 

        ```python
        completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=1200)
        ```  

        Taking this code for a spin, we now arrive at the following output:

        ```output
        レシピ数（例：5: 2
        材料リスト（例：chicken, potatoes, and carrots: apple,flour
        フィルター（例：vegetarian, vegan, or gluten-free: sugar
        レシピ:
         または milk.
        
        -アップルと小麦粉のパンケーキ: 小麦粉1カップ、ベーキングパウダー小さじ1/2、ベーキングソーダ小さじ1/2、塩小さじ1/4、砂糖大さじ1、卵1個、バターミルクまたはサワーミルク1カップ、溶かしバター1/4カップ、グラニースミスアップル1個（皮をむいてすりおろす）
        -アップルフリッター: 小麦粉1と1/2カップ、ベーキングパウダー小さじ1、塩小さじ1/4、ベーキングソーダ小さじ1/4、ナツメグ小さじ1/4、シナモン小さじ1/4、オールスパイス小さじ1/4、砂糖1/4カップ、ショートニング1/4カップ、ミルク1/4カップ、卵1個、皮をむいてすりおろしたリンゴ2カップ
        買い物リスト:
         -小麦粉、ベーキングパウダー、ベーキングソーダ、塩、砂糖、卵、バターミルク、バター、アップル、ナツメグ、シナモン、オールスパイス 
        ```
        
- **A word on token length**. We should consider how many tokens we need to generate the text we want. Tokens cost money, so where possible, we should try to be economical with the number of tokens we use. For example, can we phrase the prompt so that we can use less tokens?

   To change tokens used, you can use the `max_tokens` parameter. For example, if you want to use 100 tokens, you would do:

    ```python
    completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=100)
    ```

- **Experimenting with temperature**. Temperature is something we haven't mentioned so far but is an important context for how our program performs. The higher the temperature value the more random the output will be. Conversely the lower the temperature value the more predictable the output will be. Consider whether you want variation in your output or not.

   To alter the temperature, you can use the `temperature` parameter. For example, if you want to use a temperature of 0.5, you would do:

    ```python
    completion = client.chat.completion.create(model=deployment, messages=messages, temperature=0.5)
    ```

   > Note, 1.0に近いほど、出力のバリエーションが増えます。



## 課題

この課題では、何を作るか自分で選んでください。

いくつか提案を挙げます：

- レシピ生成アプリをさらに改良してみましょう。temperature値やプロンプトをいろいろ試してみてください。
- 「スタディバディ」を作る。例えばPythonについて質問に答えられるアプリです。「Pythonのあるトピックとは何か？」や「このトピックのコード例を見せて」などのプロンプトを用意できます。
- 歴史ボット。歴史上の人物になりきって、その人物の人生や時代について質問できるようにしましょう。

## 解答例

### スタディバディ

- 「あなたはPython言語の専門家です

    初心者向けのPythonレッスンを次のフォーマットで提案してください：
    
    フォーマット:
    - 概念:
    - レッスンの簡単な説明:
    - コードでの練習問題と解答例」

上記はスタータープロンプトです。自分なりに工夫して使ってみてください。

### 歴史ボット

次のようなプロンプトが使えます：

- 「あなたはエイブラハム・リンカーンです。自分について3文で教えてください。リンカーンらしい文法や言葉遣いで答えてください」
- 「あなたはエイブラハム・リンカーンです。リンカーンらしい文法や言葉遣いで答えてください：

   あなたの最大の功績について、300語で教えてください：」

## 知識チェック

temperatureという概念は何を制御しますか？

1. 出力のランダムさを制御する
1. レスポンスの大きさを制御する
1. 使用されるトークン数を制御する

A: 1

APIキーのような秘密情報を保存する良い方法は？

1. コード内
1. ファイル内
1. 環境変数内

A: 3。なぜなら、環境変数はコード内に保存されず、コードから読み込むことができるからです。



---

**免責事項**:  
本書類は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性には努めておりますが、自動翻訳には誤りや不正確な表現が含まれる場合があります。原文（原言語の文書）が正式な情報源と見なされるべきです。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用により生じたいかなる誤解や誤訳についても、当方は一切の責任を負いかねます。
